In [1]:
import json
import time

from pyspark.sql import SparkSession
from nltk.sentiment import SentimentIntensityAnalyzer

In [2]:
'''def classify_comments(df):
    sia = SentimentIntensityAnalyzer()
    def _classify(comment):
        scores = sia.polarity_scores(comment)
        compound = scores['compound']
        if compound >= 0.05:
            return 'positive'
        elif compound <= -0.05:
            return 'negative'
        else:
            return 'neutral'
    df['sentiment'] = df['comment_column'].apply(lambda comment: _classify(comment))
    return df

def preprocess(df):
    df['body'] = df['body'].apply(json.loads)
    df['body'] = df['body'].apply(lambda x: x['body'])
    df['body'] = df['body'].apply(lambda x: x.strip())
    return df

def analyze_comments(df):
    df_counts = df['sentiment'].value_counts().reset_index()
    df_counts.columns = ['sentiment', 'count']
    df_counts = df_counts.sort_values(by='count', ascending=False)
    return df_counts'''

In [ ]:
def preprocess(rdd):
    rdd = rdd.map(lambda line: json.loads(line))\
             .map(lambda line: line['body']) \
             .map(lambda line: line.strip())
    return rdd


def classify_comments(rdd):
    sia = SentimentIntensityAnalyzer()

    def _classify(comment):
        scores = sia.polarity_scores(comment)
        compound = scores['compound']
        if compound >= 0.05:
            return 'positive'
        elif compound <= -0.05:
            return 'negative'
        else:
            return 'neutral'

    m_rdd = rdd.map(lambda line: (line, _classify(line)))
    return m_rdd


def analyze_comments(rdd):
    rdd = rdd.map(lambda pair: pair[1]) \
             .map(lambda k: (k, 1)) \
             .reduceByKey(lambda v1, v2: v1+v2)\
             .map(lambda kv: (kv[1], kv[0])) \
             .sortByKey(False)\
             .map(lambda vk: (vk[1], vk[0]))
    return rdd


def rdd_slice(rdd, start, end):
    rdd = rdd.zipWithIndex()\
            .filter(lambda kv: kv[1] >= start and kv[1] <= end) \
            .map(lambda kv: kv[0])
    return rdd     

In [ ]:
# Parameter for the run
# Data amounts to be testing
chunk_amounts = [100, 300, 600]
# Setting for 1,2 and 4 cores
executor_cores_options = [1, 2]
executor_nodes_options = [1, 2]
# Tests per above setting
iterations = 2

spark_session = SparkSession.builder\
                           .appName(f"DF_cores_{executor_cores}_instances_{executor_nodes}_datasize_{datasize*executor_nodes*executor_cores}")\
                           .master("spark://192.168.2.230:7077")\
                           .config("spark.dynamicAllocation.enabled", True)\
                           .config("spark.executor.instances", executor_nodes)\
                           .config("spark.executor.cores", executor_cores)\
                           .config("spark.dynamicAllocation.minExecutors","1")\
                           .config("spark.dynamicAllocation.maxExecutors","1")\
                           .config("spark.driver.port", 9999)\
                           .config("spark.blockManager.port", 10005)\
                           .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [4]:
file_paths = [f"hdfs://master:9000/path/in/hdfs/JsonFiles/corpus-webis-shortened_chunk{i}.json" for i in range(1, 3844 + 1)]
df = spark_session.read.option("multiline", "true").json(file_paths)

NameError: name 'spark_session' is not defined